In [ ]:
import numpy as np
import csv
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import roc_curve
import math
import pickle

# === Constants ===
MAXSEQ = 1274
NUM_FEATURE = 20
EPOCHS = 50
BATCH_SIZE = 32
NUM_CLASSES = 2
WINDOW_SIZES = [2, 4, 8, 12, 16, 20]

# === Load Dataset ===
def load_ds(file_path):
    with open(file_path) as file:
        NUM_SAMPLES = sum(1 for _ in file)
    data = np.zeros((NUM_SAMPLES, MAXSEQ * NUM_FEATURE), dtype=np.float32)
    labels = np.zeros((NUM_SAMPLES, 1), dtype=np.uint8)
    with open(file_path) as file:
        reader = csv.reader(file, delimiter=',')
        for m, row in enumerate(reader):
            labels[m] = int(row[0])
            data[m] = np.array(row[1:]).astype('float32')
            print(f"\rReading {file_path}...\t{m+1}/{NUM_SAMPLES}", end='')
    print('\tDone')
    return data, labels

# === Prepare Data ===
x_train, y_train = load_ds('/Hussain/Efflux/train_data_A.csv')
x_test, y_test = load_ds('/Hussain/Efflux/test_data_A.csv')
x_train = x_train.reshape((-1, MAXSEQ, NUM_FEATURE, 1))
x_test = x_test.reshape((-1, MAXSEQ, NUM_FEATURE, 1))
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

# === Multi-Window CNN Model ===
def build_multi_window_cnn():
    inputs = layers.Input(shape=(MAXSEQ, NUM_FEATURE, 1))
    conv_outputs = []

    for w in WINDOW_SIZES:
        conv = layers.Conv2D(128, kernel_size=(1, w), activation='relu', padding='valid', name=f"conv_{w}")(inputs)
        pool = layers.GlobalMaxPooling2D()(conv)
        conv_outputs.append(pool)

    merged = layers.Concatenate()(conv_outputs)
    x = layers.Dense(512, activation='relu')(merged)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    return models.Model(inputs, outputs)

model = build_multi_window_cnn()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

# === Evaluation ===
pred_test = model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test_cat[:, 1], pred_test[:, 1])
AUC = metrics.auc(fpr, tpr)
gmeans = np.sqrt(tpr * (1 - fpr))
ix = np.argmax(gmeans)
threshold = thresholds[ix]
y_pred = (pred_test[:, 1] >= threshold).astype(int)

TN, FP, FN, TP = metrics.confusion_matrix(y_test_cat[:, 1], y_pred).ravel()
Sens = TP / (TP + FN) if TP + FN > 0 else 0.0
Spec = TN / (TN + FP) if TN + FP > 0 else 0.0
Acc = (TP + TN) / (TP + FP + TN + FN)
MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) if TP + FP > 0 and FP + TN > 0 and TP + FN > 0 and TN + FN > 0 else 0.0
F1 = 2 * TP / (2 * TP + FP + FN)
Prec = TP / (TP + FP) if TP + FP > 0 else 0.0
Recall = TP / (TP + FN)

print(f'\n✅ Final Evaluation Metrics:')
print(f'TP={TP}, FP={FP}, TN={TN}, FN={FN}')
print(f'Sens={Sens:.4f}, Spec={Spec:.4f}, Acc={Acc:.4f}, MCC={MCC:.4f}')
print(f'AUC={AUC:.4f}, F1={F1:.4f}, Prec={Prec:.4f}, Recall={Recall:.4f}')

# === Save ROC + AUC ===
with open("/Hussain/Efflux/auc/A_multi_window_cnn.pkl", "wb") as f:
    pickle.dump({
        "fpr": fpr,
        "tpr": tpr,
        "auc": AUC,
        "y_true": y_test_cat[:, 1],
        "y_score": pred_test[:, 1]
    }, f)

# === Grad-CAM ===
def compute_gradcam(model, img_array, class_idx, layer_name="conv_2"):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(pooled_grads[:, tf.newaxis, tf.newaxis] * conv_outputs, axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def save_gradcam(heatmap, original_input, filename, title="Grad-CAM"):
    plt.figure(figsize=(12, 4))
    plt.imshow(original_input.squeeze().T, cmap="gray", aspect="auto")
    plt.imshow(heatmap.T, cmap='jet', alpha=0.5, aspect="auto")
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()

# === Generate Grad-CAM Heatmaps ===
gradcam_output_dir = "/Hussain/Efflux/A_gradcam_multi_window"
os.makedirs(gradcam_output_dir, exist_ok=True)

for idx in range(10):
    sample = x_test[idx:idx+1]
    true_label = y_test[idx][0]
    pred_label = np.argmax(model.predict(sample))
    heatmap = compute_gradcam(model, sample, pred_label, layer_name="conv_2")  # use any: conv_2, conv_4, ...

    filename = os.path.join(gradcam_output_dir, f"sample_{idx+1}_true{true_label}_pred{pred_label}.png")
    title = f"Grad-CAM (Sample {idx+1}) - True: {true_label}, Pred: {pred_label}"
    save_gradcam(heatmap, sample, filename, title)
    print(f"✅ Saved Grad-CAM: {filename}")
